# **CS5588 Assignment 2.1**

## **Student name:** Namuun Lkhagvadorj
## **Reproducing Deep Learning or LLM Model**

**Objective:** The primary goal of our project is to predict stock prices using news articles and social media posts. To accomplish this, we need to employ Natural Language Processing (NLP) techniques to accurately assess sentiment scores from these sources. As a starting point, I implemented an LSTM model as a baseline to serve as both a foundation and a benchmark for evaluating the performance of our proposed model

I used the following dataset for training the model. Please check out the corresponding websites to explore more details about the datasets
1. FPB - financial_phrasebank https://huggingface.co/datasets/takala/
2. FIQA -Financial Opinion Mining and Question Answering https://sites.google.com/view/fiqa/home
3. TFNS -  Twitter Financial Dataset https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment

# Preparing dataset

In [ ]:

!pip install datasets transformers torch tqdm pandas huggingface_hub
!pip install sentencepiece
!pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate
!pip install torchvision torchaudio
!pip install loguru
!pip install datasets
!pip install peft
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB

In [ ]:
from datasets import load_dataset
import datasets

In [ ]:
dic = {
    0:"negative",
    1:'neutral',
    2:'positive',
}

# 1. FPB

In [ ]:
fpb_datasets = load_dataset("financial_phrasebank", "sentences_50agree")
fpb_datasets = fpb_datasets["train"]
fpb_datasets = fpb_datasets.to_pandas()
fpb_datasets.columns = ["input", "output"]
fpb_datasets["output"] = fpb_datasets["output"].apply(lambda x:dic[x])
fpb_datasets = datasets.Dataset.from_pandas(fpb_datasets)
fpb_datasets = fpb_datasets.train_test_split(seed = 42)['train']
fpb_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 3634
})

In [ ]:
train_dataset = datasets.concatenate_datasets([fpb_datasets]*6)   # we want each data source have similar number of samples
train_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 21804
})

# 2. FIQA

In [ ]:
def make_label(x):
    if x < - 0.1: return "negative"
    elif x >=-0.1 and x < 0.1: return "neutral"
    elif x >= 0.1: return "positive"

In [ ]:
dataset = load_dataset('pauri32/fiqa-2018')
dataset = datasets.concatenate_datasets([dataset["train"], dataset["validation"] ,dataset["test"] ])
dataset = dataset.to_pandas()
dataset["output"] = dataset.sentiment_score.apply(make_label)
dataset = dataset[['sentence', 'output']]
dataset.columns = ["input", "output"]
dataset = datasets.Dataset.from_pandas(dataset)
dataset = dataset.train_test_split(0.226, seed = 42)['train']
dataset

train.csv:   0%|          | 0.00/161k [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/16.7k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/961 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 938
})

In [ ]:
tmp_dataset = datasets.concatenate_datasets([dataset]*21)
train_dataset = datasets.concatenate_datasets([train_dataset, tmp_dataset])
print(tmp_dataset.num_rows)
train_dataset

19698


Dataset({
    features: ['input', 'output'],
    num_rows: 41502
})

# 3. TFNS

In [ ]:
dic = {
    0:"negative",
    1:'positive',
    2:'neutral',
}

social_media_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')
social_media_dataset = social_media_dataset['train']
social_media_dataset = social_media_dataset.to_pandas()
social_media_dataset['label'] = social_media_dataset['label'].apply(lambda x:dic[x])
social_media_dataset.columns = ['input', 'output']
social_media_dataset = datasets.Dataset.from_pandas(social_media_dataset)
social_media_dataset


tmp_dataset = datasets.concatenate_datasets([social_media_dataset]*2)
train_dataset = datasets.concatenate_datasets([train_dataset,tmp_dataset])
print(tmp_dataset.num_rows)
train_dataset

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

19086


Dataset({
    features: ['input', 'output'],
    num_rows: 60588
})

In [ ]:

all_dataset = train_dataset.shuffle(seed = 42)
all_dataset.shape

(60588, 2)

## Concatenate dataset

In [ ]:
import json
from tqdm.notebook import tqdm

In [ ]:
data_list = []
for item in all_dataset.to_pandas().itertuples():
    tmp = {}
    #tmp["instruction"] = item.instruction
    tmp["input"] = item.input
    tmp["output"] = item.output
    data_list.append(tmp)

In [ ]:
with open("dataset_new.jsonl", 'w') as f:
   for example in tqdm(data_list, desc="formatting.."):
       f.write(json.dumps(example) + '\n')

formatting..:   0%|          | 0/60588 [00:00<?, ?it/s]

# Tokenize

In [ ]:
import json
from tqdm.notebook import tqdm

import datasets
from transformers import AutoTokenizer, AutoConfig

model_name = "THUDM/chatglm2-6b"
jsonl_path = "dataset_new.jsonl"
save_path = 'dataset_new'
max_seq_length = 512
skip_overlength = True

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

def read_jsonl(path, max_seq_length, skip_overlength=False):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

In [ ]:
import json
import csv

# Open the JSONL file for reading
with open('/content/dataset_new.jsonl', 'r') as jsonl_file:
    # Open a CSV file for writing
    with open('output.csv', 'w', newline='') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)

        # Iterate over each line in the JSONL file
        for line in jsonl_file:
            # Parse each line as a separate JSON object
            try:
                data = json.loads(line)

                # Extract the desired fields (replace with actual field names)
                field1 = data.get('input', '')
                field2 = data.get('output', '')

                # Write the data to the CSV file
                csv_writer.writerow([field1, field2])

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

In [ ]:
import pandas as pd
df = pd.read_csv('output.csv')
df.columns = ['input', 'output']
df['output'] = df['output'].apply(lambda x:x.strip())
df

,input,output
0,We're Keeping An Eye On Sangamo Therapeutics's...,neutral
1,The company 's share is quoted on NASDAQ OMX H...,neutral
2,China Mengniu Dairy : DISCLOSEABLE TRANSACTION...,neutral
3,The acquisition price was not disclosed .,neutral
4,Lagarde Says ECB's Options Limited by Low Rate...,negative
...,...,...
60582,"BP, Statoil, to Withdraw Staff From Algeria Fo...",negative
60583,"The copying , republication or redistribution ...",neutral
60584,Operating profit margin increased from 11.2 % ...,positive
60585,$vxx adding to position here !,positive


# Modelling part - LSTM

In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from tensorflow.keras.preprocessing.text
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

# Data preprocessing

In [ ]:
def remove_tags(string):
    # Remove HTML tags
    result = re.sub(r'<[^>]+>', '', string)
    # Remove URLs
    result = re.sub(r'https?://\S+|www\.\S+', '', result)
    # Remove non-alphanumeric characters (keeping spaces)
    result = re.sub(r'[^a-zA-Z0-9\s]', ' ', result)
    # Convert to lowercase
    result = result.lower()
    # Remove extra spaces (optional)
    result = re.sub(r'\s+', ' ', result).strip()
    return result

# Apply the function to the 'input' column of the DataFrame
df['input'] = df['input'].apply(remove_tags)


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['input'] = df['input'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['input'] = df.input.apply(lemmatize_text)
df

[nltk_data] Downloading package wordnet to /root/nltk_data...


,input,output
0,keeping eye sangamo therapeutic nasdaq sgmo ca...,neutral
1,company share quoted nasdaq omx helsinki rauta...,neutral
2,china mengniu dairy discloseable transaction a...,neutral
3,acquisition price disclosed,neutral
4,lagarde say ecb option limited low rate low in...,negative
...,...,...
60582,bp statoil withdraw staff algeria following ro...,negative
60583,copying republication redistribution afx news ...,neutral
60584,operating profit margin increased 11 2 11 7,positive
60585,vxx adding position,positive


## Encoding labels and splitting training dataset

In [ ]:
reviews = df['input'].values
labels = df['output'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [ ]:
# Splitting training and testing dataset
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

# Model Building

## Parameters for LSTM

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

**Note:** An embedding layer of dimension 100 converts each word in the sentence into a fixed-length dense vector of size 100. The input dimension is the vocabulary size, and the output dimension is 100. Hence, each word in the input will be represented by a vector of size 100.
A bidirectional LSTM layer of 64 units.
A dense (fully connected) layer of 24 units with relu activation.
A dense layer of 1 unit and sigmoid activation outputs the probability of the review is positive, i.e., if the label is 1.

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.build(input_shape=(None, 200))
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 100)            │         300,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          84,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 24)                  │           3,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 387,601 (1.48 MB)

 Trainable params: 387,601 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define model
model = keras.Sequential([
    # Embedding layer
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),

    # Bidirectional LSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),

    # Additional LSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),

    # Global max pooling layer to reduce dimensions
    keras.layers.GlobalMaxPooling1D(),

    # Dense hidden layer
    keras.layers.Dense(64, activation='relu'),

    # Dropout layer for regularization
    keras.layers.Dropout(0.5),

    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])

# Build the model
model.build(input_shape=(None, max_length))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 200, 100)            │         300,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 200, 128)            │          84,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 200, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 128)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 491,617 (1.88 MB)

 Trainable params: 491,617 (1.88 MB)

 Non-trainable params: 0 (0.00 B)

# Model training and Evaluation

In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/5
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 44s 30ms/step - accuracy: 0.4533 - loss: -155.8432 - val_accuracy: 0.4608 - val_loss: -2153.2153
Epoch 2/5
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.5056 - loss: -3836.8767 - val_accuracy: 0.4930 - val_loss: -9799.1592
Epoch 3/5
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.5141 - loss: -12920.5303 - val_accuracy: 0.5328 - val_loss: -20907.3008
Epoch 4/5
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.5170 - loss: -25796.6133 - val_accuracy: 0.5117 - val_loss: -36700.1133
Epoch 5/5
1278/1278 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step - accuracy: 0.5145 - loss: -43541.1992 - val_accuracy: 0.5222 - val_loss: -54655.8438


In [ ]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

474/474 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step
Accuracy of prediction on test set :  0.5224136792764243
